数据标准化并不复杂，两三行代码就能避免很多问题，因此，对一些量纲相差较大的特征变量，实战中通常会先进行数据标准化，再进行训练集和测试集划分。除了K近邻算法模型，还有一些模型也是基于距离的，所以量纲对模型影响较大，就需要进行数据标准化，如支持向量机模型、KMeans聚类分析、PCA（主成分分析）等。此外，对于一些线性模型，如第3章的线性回归模型和第4章的逻辑回归模型，有时也需要进行数据标准化处理。对于树模型则无须做数据标准化处理，因为数值缩放不影响分裂点位置，对树模型的结构不造成影响。因此，决策树模型及基于决策树模型的随机森林模型、AdaBoost模型、GBDT模型、XGBoost模型、LightGBM模型通常都不需要进行数据标准化处理，因为它们不关心变量的值，而是关心变量的分布和变量之间的条件概率。笔者验证过（具体见10.3.4小节的“补充知识点”），在树模型相关的机器学习模型中，进行数据标准化对预测结果不会产生影响。在实际工作中，如果不确定是否要做数据标准化，可以先尝试做一做数据标准化，看看模型预测准确度是否有提升，如果提升较明显，则推荐进行数据标准化。

> 实战中等宽分箱应用相对较多

In [3]:
# 构造演示数据
import pandas as pd
data = pd.DataFrame([[22,1],[25,1],[20,0],[35,0],[32,1],[38,0],[50,0],
[46,1]], columns=['年龄', '是否违约'])
data

,年龄,是否违约
0,22,1
1,25,1
2,20,0
3,35,0
4,32,1
5,38,0
6,50,0
7,46,1


In [5]:
# 对“年龄”这一特征变量进行等宽分箱
data_cut = pd.cut(data['年龄'], 3)
data_cut

0    (19.97, 30.0]
1    (19.97, 30.0]
2    (19.97, 30.0]
3     (30.0, 40.0]
4     (30.0, 40.0]
5     (30.0, 40.0]
6     (40.0, 50.0]
7     (40.0, 50.0]
Name: 年龄, dtype: category
Categories (3, interval[float64, right]): [(19.97, 30.0] < (30.0, 40.0] < (40.0, 50.0]]

“年龄”列中数据的范围是20～50岁，分为3组恰好为20～30岁（19.97近似为20）、30～40岁、40～50岁，可以看到，每个分箱的年龄差都是10岁，这就是等宽分箱。

In [11]:
# 用groupby()函数进行分组，用count()函数进行计数，可以获取每个分箱中的样本数
n = data['是否违约'].groupby(data_cut).count()
print(n)

年龄
(19.97, 30.0]    3
(30.0, 40.0]     3
(40.0, 50.0]     2
Name: 是否违约, dtype: int64


信用评分卡模型可以依据年龄、月收入、负债比率、历史逾期次数等多个特征变量对客户进行评估。对特征变量年龄进行离散化可以使建立的模型更稳定，例如，将20～30岁作为一个类别，如果客户从25岁增长为26岁，也不会因此成为完全不同类别的人。但是，年龄位于类别区间边界点的客户则会因为增长1岁而被分到另一个类别，因此，在分箱时要谨慎选取类别间的界限。数据分箱还有一个好处就是可以剔除异常值的影响，也是异常值处理的一个手段。

#### 学习数据分箱主要是学习WOE值与IV值做准备，计算WOE值的第一个步骤就是数据分箱。